In [1]:
import os
import psutil
import signal

import numpy
import sympy
import mpmath
import itertools
import pandas as pd
import multiprocessing as mp
from datetime import datetime

import MAE_funcs

from sympy import *
from sympy.abc import *

from scipy.optimize import minimize

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [37]:
import multiprocessing as mp
import MAE_funcs
import warnings
import os
import tqdm
import random
warnings.filterwarnings("ignore")
import os
import psutil
import signal

import numpy
import sympy
import mpmath
import itertools
import pandas as pd
import multiprocessing as mp
from datetime import datetime

import MAE_funcs

from sympy import *
from sympy.abc import *

from scipy.optimize import minimize

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



params = {'H': [], 'Ms': [], 'X': [], 't': [], 'td': [], 'f': [], 'nu': [], 'g': []}
params['H']  = [10**((i-4)*2) for i in range(0, 12+1, 2)]
params['Ms'] = [10**i for i in range(1,3+1)]
params['X']  = [1000**(i-2) for i in range(1,3+1)]
params['t']  = [(0.7057 - 0.99*i)/(-29.4286 + i) for i in range(1, 29+1, 2)]
params['td'] = [(0.7057 - 0.99*i)/(-29.4286 + i) for i in range(1, 29+1, 2)]
params['f']  = [0.005+(i-1)*.07 for i in range(1, 15+1, 2)]
params['nu'] = [.245*i-.24 for i in range(1, 15+1, 2)]
params['g']  = [10**7]
paramList = list(itertools.product(params['H'], params['Ms'], params['X'],  params['t'], params['td'], params['f'], params['nu'], params['g']))
params = {'H': [], 'Ms': [], 'X': [], 't': [], 'td': [], 'f': [], 'nu': [], 'g': []}
_keys = list(params.keys())
for idx in range(len(paramList[0])):
    params[_keys[idx]] = [_value[idx] for _value in paramList]
paramDF = pd.DataFrame(data=params)



def main(paramList, numRuns, rand):
    #pseudo batching using pool here
    #since 1 minimization per job is slower
    #send multiple minimizations to each job, which is faster
    #since multiple processes saves previous info in cache
    
    #little randmization option
    if rand:
        newList = []
        for pick in paramList:
            rand = random.randint(0, len(paramList)-1)
            newList.append(paramList[rand])
        paramList = newList
    
    splitList = numpy.array_split(paramList[:numRuns], int(numRuns*.1))
    #import pdb; pdb.set_trace()
    print('#jobs: ' + str(len(paramList[:numRuns])))
    pool = mp.Pool()
    try:
        #results = pool.imap(MAE_funcs.FreeEnergyTI, paramList[:num])
        results = list(tqdm.tqdm(pool.imap(MAE_funcs.main, splitList)))
        pool.close()
    except KeyboardInterrupt:
        pool.terminate()        
    pool.join()

    r = []
    for result in results:
        r.append(result)
    print('#finished: ' + str(len(r)))
    
    return r


    
if __name__ == "__main__":
    start = datetime.now()
    out = main(paramList = paramList, numRuns = 100000, rand = True)
    print(datetime.now() - start)
    

0it [00:00, ?it/s]

#jobs: 100000


10000it [07:04, 23.58it/s]


#finished: 10000
0:07:05.835929


In [56]:
allOut = [item for sublist in out for item in sublist]

In [63]:
clean = []
for entry in allOut:
    if entry[-1] < 1 and entry[-2] < 1 and entry[-3] < 1:
        if entry[-1] != 0 and entry[-2] != 0 and entry[-3] != 0:
            clean.append(entry)

In [4]:
MAE_funcs.FreeEnergyTI((100000,1000,1000,10.7159,10.7159,0.845,0.495,10000000))

defining params
defining variables
defining matrices
	 H:   100000
	 EH3: 100000.0
minimizing
[100000, 1000, 1000, 10.7159, 10.7159, 0.845, 0.495, 10000000, 0.00019041895142018198, -0.00016088735415987747, -3.225865976454756e-05]


C:\Users\rathj\anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


[100000,
 1000,
 1000,
 10.7159,
 10.7159,
 0.845,
 0.495,
 10000000,
 0.00019041895142018198,
 -0.00016088735415987747,
 -3.225865976454756e-05]

In [4]:
%debug

> c:\users\rathj\mae\mae_funcs.py(56)EH3()
     54             print('Warning, solving for EH3 gives more than 1 solution: ' + str(EH3))
     55             raise
---> 56         EH3 = float(EH3[0])
     57         print('\t H:   ' + str(H))
     58         print('\t EH3: ' + str(EH3))



ipdb>  EH3


[]


ipdb>  sympy.solvers.solve(EH_eq)


[]


ipdb>  q


In [39]:
#SYMBOLIC#
#assigning symbols
z, Sigma, g, nu, k, l, n, m, p, k1, h0, h, t,td, kp, lp, np, mp, pp, kd, ld, nd, md, pd, L1, Pi, Pd, P, Ltilde, ex, ey, ez, a, f, H, D, M, e, C, Mag, Ms, MTI, EH, EH3, Lz, X, ax, ay, az = symbols('z Sigma g nu k l n m p k1 h0 h t td kp lp np mp pp kd ld nd md pd L1 Pi Pd P Ltilde ex ey ez a f H D M e C Mag Ms MTI EH EH3, Lz X ax ay az', real=True)

Sigma = 2*g*(1+nu)
k     = Sigma / (2*(1+nu)*(1-2*nu))
l     = (Sigma*nu) / (2*(1+nu)*(1-2*nu))
n     = Sigma*(1-nu) / (2*(1+nu)*(1-2*nu))
m     = Sigma / (2*(1+nu))
p     = Sigma / (2*(1+nu))
k1    = 2*g*(1+nu) / (1-2*nu)
kp    = ((7*h0-2*t**2-4*t**2*h0)*g+3*(h0-2*t**2+2*t**2*h0)*k1) / (8*(1-t**2)*g*(4*g+3*k1))
lp    = ((g+3*k1)*(2*t**2-h0-2*t**2*h0)) / (4*(1-t**2)*g*(4*g+3*k1))
np    = ((6-5*h0-8*t**2+8*t**2*h0)*g+3*(h0-2*t**2+2*t**2*h0)*k1) / (2*(1-t**2)*g*(4*g+3*k1))
mp    = ((15*h0-2*t**2-12*t**2*h0)*g+3*(3*h0-2*t**2)*k1) / (16*(1-t**2)*g*(4*g+3*k1))
pp    = (2*(4-3*h0-2*t**2)*g+3*(2-3*h0+2*t**2-3*t**2*h0)*k1) / (8*(1-t**2)*g*(4*g+3*k1))
kd    = ((7*h-2*td**2-4*td**2*h)*g+3*(h-2*td**2+2*td**2*h)*k1) / (8*(1-td**2)*g*(4*g+3*k1))
ld    = ((g+3*k1)*(2*td**2-h-2*td**2*h)) / (4*(1-td**2)*g*(4*g+3*k1))
nd    = ((6-5*h-8*td**2+8*td**2*h)*g+3*(h-2*td**2+2*td**2*h)*k1) / (2*(1-td**2)*g*(4*g+3*k1))
md    = ((15*h-2*td**2-12*td**2*h)*g+3*(3*h-2*td**2)*k1) / (16*(1-td**2)*g*(4*g+3*k1))
pd    = (2*(4-3*h-2*td**2)*g+3*(2-3*h+2*td**2-3*td**2*h)*k1) / (8*(1-td**2)*g*(4*g+3*k1))

Lx = Function('Lx')(ex, ey, ez)
Ly = Function('Ly')(ex, ey, ez)
Lz = Function('Lz')(ex, ey, ez)

#Lx: a is (ex)/(ez)
#Ly: a is (ey)/(ez)
#Lz: a is (ez)/(ex(or ey)) 
Lreplace = [
(Lx, 1-1/(1-ax**2)*(1-ax/sqrt(1-ax**2)*asin(sqrt(1-ax**2)))/2),
(Ly, 1-1/(1-ay**2)*(1-ay/sqrt(1-ay**2)*asin(sqrt(1-ay**2)))/2),
(Lz, 1/(1-az**2)*(1-az/sqrt(1-az**2)*asin(sqrt(1-az**2)))),
(ax, td*((1+ex)/(1+ez))),
(ay, td*((1+ey)/(1+ez))),
(az, td*((1+ez)/(1+ex)))
]
L1 = SparseMatrix([[k1+(4*g)/3, k1-(2*g)/3, k1-(2*g)/3, 0, 0, 0], [k1-2*g/3, k1+4*g/3, k1-2*g/3, 0, 0, 0], [k1-2*g/3, k1-2*g/3, k1+4*g/3, 0, 0, 0], [0, 0, 0, g, 0, 0], [0, 0, 0, 0, g, 0], [0, 0, 0, 0, 0, g]])
Pi = SparseMatrix([[kp+mp, kp-mp, lp, 0, 0, 0], [kp-mp, kp+mp, lp, 0, 0, 0], [lp, lp, np, 0, 0, 0], [0, 0, 0, 2*pp, 0, 0], [0, 0, 0, 0, 2*pp, 0], [0, 0, 0, 0, 0, 2*mp]])
Pd = SparseMatrix([[kd+md, kd-md, ld, 0, 0, 0], [kd-md, kd+md, ld, 0, 0, 0], [ld, ld, nd, 0, 0, 0], [0, 0, 0, 2*pd, 0, 0], [0, 0, 0, 0, 2*pd, 0], [0, 0, 0, 0, 0, 2*md]])
P = 1/(1-f)*(Pi-f*Pd)
#Ltilde = L1+f/(1-f)*P.inv()

#D = re((4*numpy.pi)/(t**2-1)*(t/(2*(t**2-1)**(1/2))*log((t+(t**2-1)**(1/2))/(t-(t**2-1)**(1/2)))-1))
#C = SparseMatrix([[Ltilde[0,0], Ltilde[0,1], Ltilde[0,2]], [Ltilde[1,0], Ltilde[1,1], Ltilde[1,2]], [Ltilde[2,0], Ltilde[2,1], Ltilde[2,2]]])
#MTI = (SparseMatrix([[ex, ey, ez]]) * C * SparseMatrix([[ex], [ey], [ez]]))[0]

In [41]:
param = paramList[100]
H  = param[0]
Ms = param[1]
X  = param[2]
t  = param[3]
td = param[4]
f  = param[5]
nu = param[6]
g  = param[7]
h  = _h(td)
h0 = _h0(t)
start = datetime.now()
print(datetime.now() - start)

0:00:00


In [1]:
from datetime import datetime
start = datetime.now()
from sympy import *
from sympy.abc import *

z, Sigma, g, nu, k, l, n, m, p, k1, h0, h, t,td, kp, lp, np, mp, pp, kd, ld, nd, md, pd, L1, Pi, Pd, P, Ltilde, ex, ey, ez, a, f, H, D, M, e, C, Mag, Ms, MTI, EH, EH3, Lz, X, ax, ay, az = symbols('z Sigma g nu k l n m p k1 h0 h t td kp lp np mp pp kd ld nd md pd L1 Pi Pd P Ltilde ex ey ez a f H D M e C Mag Ms MTI EH EH3, Lz X ax ay az', real=True)
Sigma = 2*g*(1+nu)
k     = Sigma / (2*(1+nu)*(1-2*nu))
l     = (Sigma*nu) / (2*(1+nu)*(1-2*nu))
n     = Sigma*(1-nu) / (2*(1+nu)*(1-2*nu))
m     = Sigma / (2*(1+nu))
p     = Sigma / (2*(1+nu))
k1    = 2*g*(1+nu) / (1-2*nu)
kp    = ((7*h0-2*t**2-4*t**2*h0)*g+3*(h0-2*t**2+2*t**2*h0)*k1) / (8*(1-t**2)*g*(4*g+3*k1))
lp    = ((g+3*k1)*(2*t**2-h0-2*t**2*h0)) / (4*(1-t**2)*g*(4*g+3*k1))
np    = ((6-5*h0-8*t**2+8*t**2*h0)*g+3*(h0-2*t**2+2*t**2*h0)*k1) / (2*(1-t**2)*g*(4*g+3*k1))
mp    = ((15*h0-2*t**2-12*t**2*h0)*g+3*(3*h0-2*t**2)*k1) / (16*(1-t**2)*g*(4*g+3*k1))
pp    = (2*(4-3*h0-2*t**2)*g+3*(2-3*h0+2*t**2-3*t**2*h0)*k1) / (8*(1-t**2)*g*(4*g+3*k1))
kd    = ((7*h-2*td**2-4*td**2*h)*g+3*(h-2*td**2+2*td**2*h)*k1) / (8*(1-td**2)*g*(4*g+3*k1))
ld    = ((g+3*k1)*(2*td**2-h-2*td**2*h)) / (4*(1-td**2)*g*(4*g+3*k1))
nd    = ((6-5*h-8*td**2+8*td**2*h)*g+3*(h-2*td**2+2*td**2*h)*k1) / (2*(1-td**2)*g*(4*g+3*k1))
md    = ((15*h-2*td**2-12*td**2*h)*g+3*(3*h-2*td**2)*k1) / (16*(1-td**2)*g*(4*g+3*k1))
pd    = (2*(4-3*h-2*td**2)*g+3*(2-3*h+2*td**2-3*td**2*h)*k1) / (8*(1-td**2)*g*(4*g+3*k1))

L1 = SparseMatrix([[k1+(4*g)/3, k1-(2*g)/3, k1-(2*g)/3, 0, 0, 0], [k1-2*g/3, k1+4*g/3, k1-2*g/3, 0, 0, 0], [k1-2*g/3, k1-2*g/3, k1+4*g/3, 0, 0, 0], [0, 0, 0, g, 0, 0], [0, 0, 0, 0, g, 0], [0, 0, 0, 0, 0, g]])
Pi = SparseMatrix([[kp+mp, kp-mp, lp, 0, 0, 0], [kp-mp, kp+mp, lp, 0, 0, 0], [lp, lp, np, 0, 0, 0], [0, 0, 0, 2*pp, 0, 0], [0, 0, 0, 0, 2*pp, 0], [0, 0, 0, 0, 0, 2*mp]])
Pd = SparseMatrix([[kd+md, kd-md, ld, 0, 0, 0], [kd-md, kd+md, ld, 0, 0, 0], [ld, ld, nd, 0, 0, 0], [0, 0, 0, 2*pd, 0, 0], [0, 0, 0, 0, 2*pd, 0], [0, 0, 0, 0, 0, 2*md]])
P = 1/(1-f)*(Pi-f*Pd)


print(datetime.now() - start)

0:00:00.436388


In [ ]:
start = datetime.now()
print(datetime.now() - start)

In [4]:
start = datetime.now()

import numpy

param = paramList[100]
H  = param[0]
Ms = param[1]
X  = param[2]
t  = param[3]
td = param[4]
f  = param[5]
nu = param[6]
g  = param[7]
h  = _h(td)
h0 = _h0(t)

Sigma = 2*g*(1+nu)
k     = Sigma / (2*(1+nu)*(1-2*nu))
l     = (Sigma*nu) / (2*(1+nu)*(1-2*nu))
n     = Sigma*(1-nu) / (2*(1+nu)*(1-2*nu))
m     = Sigma / (2*(1+nu))
p     = Sigma / (2*(1+nu))
k1    = 2*g*(1+nu) / (1-2*nu)
kp    = ((7*h0-2*t**2-4*t**2*h0)*g+3*(h0-2*t**2+2*t**2*h0)*k1) / (8*(1-t**2)*g*(4*g+3*k1))
lp    = ((g+3*k1)*(2*t**2-h0-2*t**2*h0)) / (4*(1-t**2)*g*(4*g+3*k1))
np    = ((6-5*h0-8*t**2+8*t**2*h0)*g+3*(h0-2*t**2+2*t**2*h0)*k1) / (2*(1-t**2)*g*(4*g+3*k1))
mp    = ((15*h0-2*t**2-12*t**2*h0)*g+3*(3*h0-2*t**2)*k1) / (16*(1-t**2)*g*(4*g+3*k1))
pp    = (2*(4-3*h0-2*t**2)*g+3*(2-3*h0+2*t**2-3*t**2*h0)*k1) / (8*(1-t**2)*g*(4*g+3*k1))
kd    = ((7*h-2*td**2-4*td**2*h)*g+3*(h-2*td**2+2*td**2*h)*k1) / (8*(1-td**2)*g*(4*g+3*k1))
ld    = ((g+3*k1)*(2*td**2-h-2*td**2*h)) / (4*(1-td**2)*g*(4*g+3*k1))
nd    = ((6-5*h-8*td**2+8*td**2*h)*g+3*(h-2*td**2+2*td**2*h)*k1) / (2*(1-td**2)*g*(4*g+3*k1))
md    = ((15*h-2*td**2-12*td**2*h)*g+3*(3*h-2*td**2)*k1) / (16*(1-td**2)*g*(4*g+3*k1))
pd    = (2*(4-3*h-2*td**2)*g+3*(2-3*h+2*td**2-3*td**2*h)*k1) / (8*(1-td**2)*g*(4*g+3*k1))

L1 = numpy.matrix([[k1+(4*g)/3, k1-(2*g)/3, k1-(2*g)/3, 0, 0, 0], [k1-2*g/3, k1+4*g/3, k1-2*g/3, 0, 0, 0], [k1-2*g/3, k1-2*g/3, k1+4*g/3, 0, 0, 0], [0, 0, 0, g, 0, 0], [0, 0, 0, 0, g, 0], [0, 0, 0, 0, 0, g]])
Pi = numpy.matrix([[kp+mp, kp-mp, lp, 0, 0, 0], [kp-mp, kp+mp, lp, 0, 0, 0], [lp, lp, np, 0, 0, 0], [0, 0, 0, 2*pp, 0, 0], [0, 0, 0, 0, 2*pp, 0], [0, 0, 0, 0, 0, 2*mp]])
Pd = numpy.matrix([[kd+md, kd-md, ld, 0, 0, 0], [kd-md, kd+md, ld, 0, 0, 0], [ld, ld, nd, 0, 0, 0], [0, 0, 0, 2*pd, 0, 0], [0, 0, 0, 0, 2*pd, 0], [0, 0, 0, 0, 0, 2*md]])
#P = 1/(1-f)*(Pi-f*Pd)


print(datetime.now() - start)

0:00:00.001001


In [3]:
def _h0(t):
    if t<1:
        h0 = (t*(numpy.arccos(t)-t*(1-t**2)**(1/2))) / (1-t**2)**(3/2)
    if t>1:
        h0 = (re(t*(-numpy.arccosh(t)+t*(t**2-1)**(1/2))) / (-1+t**2)**(3/2))
    if td == 1:
        h0 = 2/3
    return(h0)

def _h(td):
    if td<1:
        h  = (td*(numpy.arccos(td)-td*(1-td**2)**(1/2))) / (1-td**2)**(3/2)
    if td>1:
        h  = (re(td*(-numpy.arccosh(td)+td*(td**2 -1)**(1/2))) / (-1+td**2)**(3/2))
    if td == 1:
        h  = 2/3
    return(h)

In [31]:
def Mag(H, Ms, X, D, f, t, _td, EH):
    #Magnitization depends on effective H (EH)
    Mag = Ms * tanh(X * EH / Ms)
    return Mag

def EH3(H, Ms, X, D, f, t, _td):
    #finding root of given EH equation at H
    #we format into mpmath to use its findroot function
    _ex = 0
    _ey = 0
    _ez = 0
    eReplace = [(ex, _ex), (ey, _ey), (ez, _ez), (td, _td)]
    EH_eq = H-D*Mag(H, Ms, X, D, f, t, td, EH) + f*Lz.subs(Lreplace).subs(eReplace)*Mag(H, Ms, X, D, f, t, td, EH) / ((1+_ex)*(1+_ey)*(1+_ez))
    #lam_EH3 = lambdify(EH, EH_eq, 'mpmath')
    #EH3 = mpmath.findroot(lam_EH3, 100, solver = 'mdnewton')
    EH3 = sympy.solvers.solve(EH_eq)
    if len(EH3) > 1:
        print('Warning, solving for EH3 gives more than 1 solution: ' + EH3)
        raise
    EH3 = EH3[0]
    return EH3

In [39]:
def FreeEnergyTI(param):
    #print(param)
    #TI: transverse isotropy
    H, Ms, X, t, td, f, nu, g, h, h0 = symbols('H, Ms, X, t, td, f, nu, g, h, h0', real = True)
    z, Sigma, g, nu, k, l, n, m, p, k1, h0, h, t,td, kp, lp, np, mp, pp, kd, ld, nd, md, pd, L1, Pi, Pd, P, Ltilde, ex, ey, ez, a, f, H, D, M, e, C, Ms, MTI, EH, Lz, X, ax, ay, az = symbols('z, Sigma g nu k l n m p k1 h0 h t td kp lp np mp pp kd ld nd md pd L1 Pi Pd P Ltilde ex ey ez a f H D M e C Ms MTI EH Lz X ax ay az', real=True)
    #print('defining params')
    param = list(param)
    H  = param[0]
    Ms = param[1]
    X  = param[2]
    t  = param[3]
    td = param[4]
    f  = param[5]
    nu = param[6]
    g  = param[7]
    h  = _h(td)
    h0 = _h0(t)

    #print('defining variables')
    Sigma = 2*g*(1+nu)
    k     = Sigma / (2*(1+nu)*(1-2*nu))
    l     = (Sigma*nu) / (2*(1+nu)*(1-2*nu))
    n     = Sigma*(1-nu) / (2*(1+nu)*(1-2*nu))
    m     = Sigma / (2*(1+nu))
    p     = Sigma / (2*(1+nu))
    k1    = 2*g*(1+nu) / (1-2*nu)
    kp    = ((7*h0-2*t**2-4*t**2*h0)*g+3*(h0-2*t**2+2*t**2*h0)*k1) / (8*(1-t**2)*g*(4*g+3*k1))
    lp    = ((g+3*k1)*(2*t**2-h0-2*t**2*h0)) / (4*(1-t**2)*g*(4*g+3*k1))
    np    = ((6-5*h0-8*t**2+8*t**2*h0)*g+3*(h0-2*t**2+2*t**2*h0)*k1) / (2*(1-t**2)*g*(4*g+3*k1))
    mp    = ((15*h0-2*t**2-12*t**2*h0)*g+3*(3*h0-2*t**2)*k1) / (16*(1-t**2)*g*(4*g+3*k1))
    pp    = (2*(4-3*h0-2*t**2)*g+3*(2-3*h0+2*t**2-3*t**2*h0)*k1) / (8*(1-t**2)*g*(4*g+3*k1))
    kd    = ((7*h-2*td**2-4*td**2*h)*g+3*(h-2*td**2+2*td**2*h)*k1) / (8*(1-td**2)*g*(4*g+3*k1))
    ld    = ((g+3*k1)*(2*td**2-h-2*td**2*h)) / (4*(1-td**2)*g*(4*g+3*k1))
    nd    = ((6-5*h-8*td**2+8*td**2*h)*g+3*(h-2*td**2+2*td**2*h)*k1) / (2*(1-td**2)*g*(4*g+3*k1))
    md    = ((15*h-2*td**2-12*td**2*h)*g+3*(3*h-2*td**2)*k1) / (16*(1-td**2)*g*(4*g+3*k1))
    pd    = (2*(4-3*h-2*td**2)*g+3*(2-3*h+2*td**2-3*td**2*h)*k1) / (8*(1-td**2)*g*(4*g+3*k1))

    Lx = Function('Lx')(ex, ey, ez)
    Ly = Function('Ly')(ex, ey, ez)
    Lz = Function('Lz')(ex, ey, ez)

    #Lx: a is (ex)/(ez)
    #Ly: a is (ey)/(ez)
    #Lz: a is (ez)/(ex(or ey)) 
    Lreplace = [
    (Lx, 1-1/(1-ax**2)*(1-ax/sqrt(1-ax**2)*asin(sqrt(1-ax**2)))/2),
    (Ly, 1-1/(1-ay**2)*(1-ay/sqrt(1-ay**2)*asin(sqrt(1-ay**2)))/2),
    (Lz, 1/(1-az**2)*(1-az/sqrt(1-az**2)*asin(sqrt(1-az**2)))),
    (ax, td*((1+ex)/(1+ez))),
    (ay, td*((1+ey)/(1+ez))),
    (az, td*((1+ez)/(1+ex)))
    ]
    
    #print('defining matrices')
    L1 = SparseMatrix([[k1+(4*g)/3, k1-(2*g)/3, k1-(2*g)/3, 0, 0, 0], [k1-2*g/3, k1+4*g/3, k1-2*g/3, 0, 0, 0], [k1-2*g/3, k1-2*g/3, k1+4*g/3, 0, 0, 0], [0, 0, 0, g, 0, 0], [0, 0, 0, 0, g, 0], [0, 0, 0, 0, 0, g]])
    Pi = SparseMatrix([[kp+mp, kp-mp, lp, 0, 0, 0], [kp-mp, kp+mp, lp, 0, 0, 0], [lp, lp, np, 0, 0, 0], [0, 0, 0, 2*pp, 0, 0], [0, 0, 0, 0, 2*pp, 0], [0, 0, 0, 0, 0, 2*mp]])
    Pd = SparseMatrix([[kd+md, kd-md, ld, 0, 0, 0], [kd-md, kd+md, ld, 0, 0, 0], [ld, ld, nd, 0, 0, 0], [0, 0, 0, 2*pd, 0, 0], [0, 0, 0, 0, 2*pd, 0], [0, 0, 0, 0, 0, 2*md]])
    P = 1/(1-f)*(Pi-f*Pd)
    Ltilde = L1+f/(1-f)*P.inv()

    D = re((4*numpy.pi)/(t**2-1)*(t/(2*(t**2-1)**(1/2))*log((t+(t**2-1)**(1/2))/(t-(t**2-1)**(1/2)))-1))
    C = SparseMatrix([[Ltilde[0,0], Ltilde[0,1], Ltilde[0,2]], [Ltilde[1,0], Ltilde[1,1], Ltilde[1,2]], [Ltilde[2,0], Ltilde[2,1], Ltilde[2,2]]])
    MTI = (SparseMatrix([[ex, ey, ez]]) * C * SparseMatrix([[ex], [ey], [ez]]))[0]

    #Equation for free energy equation in transverse isotropy. Reference NIST notebook for equation.
    FTI = -f*H*Mag(H, Ms, X, D, f, t, td, EH3(H, Ms, X, D, f, t, td)) + ((f*D*Mag(H, Ms, X, D, f, t, td, EH3(H, Ms, X, D, f, t, td))**2)/2)*((Lx.subs(Lreplace) + Ly.subs(Lreplace) + Lz.subs(Lreplace))/((1+ex)*(1+ey)*(1+ez))) + MTI/2
    
    #print('minimizing')
    strains = (FTI.free_symbols)
    FTI_lam = lambdify(strains, FTI, modules = 'numpy')
    def vectorizeFTI(_list):
        return FTI_lam(_list[0], _list[1], _list[2])
    strains0 = numpy.array([0,0,0])
    minimization = minimize(fun = vectorizeFTI, x0 = strains0, method = 'Nelder-Mead');
    
    return list(minimization.x)

In [ ]:
Lreplace = [
(Lx, 1-1/(1-a**2)*(1-a/sqrt(1-a**2)*asin(sqrt(1-a**2)))/2),
(Ly, 1-1/(1-a**2)*(1-a/sqrt(1-a**2)*asin(sqrt(1-a**2)))/2),
(Lz, 1/(1-a**2)*(1-a/sqrt(1-a**2)*asin(sqrt(1-a**2))))
]

#Lofland Derivations
d1replace = [
(diff(Lx, ex), -(a**2*(-3*sqrt(-1+a**2))+(2+a**2)*asin(sqrt(1-1/a**2)))/(2*(-1+a**2)**(5/2))),
(diff(Lx, ey), (a*(-3*a*sqrt(-1+a**2))+(1+2*a**2)*log(a+sqrt(-1+a**2)))/(2*(-1+a**2)**(5/2)) + (a**2*(-3*sqrt(-1+a**2))+(2+a**2)*asin(sqrt(1-1/a**2)))/(2*(-1+a**2)**(5/2))),
(diff(Lx, ez), -(a*(-3*a*sqrt(-1+a**2))+(1+2*a**2)*log(a+sqrt(-1+a**2)))/(2*(-1+a**2)**(5/2))),
(diff(Ly, ex), (a*(-3*a*sqrt(-1+a**2))+(1+2*a**2)*log(a+sqrt(-1+a**2)))/(2*(-1+a**2)**(5/2)) + (a**2*(-3*sqrt(-1+a**2))+(2+a**2)*asin(sqrt(1-1/a**2)))/(2*(-1+a**2)**(5/2))),
(diff(Ly, ey), -(a**2*(-3*sqrt(-1+a**2))+(2+a**2)*asin(sqrt(1-1/a**2)))/(2*(-1+a**2)**(5/2))),
(diff(Ly, ez), -(a*(-3*a*sqrt(-1+a**2))+(1+2*a**2)*log(a+sqrt(-1+a**2)))/(2*(-1+a**2)**(5/2))),
(diff(Lz, ex), (a*(-3*a*sqrt(-1+a**2))+(1+2*a**2)*log(a+sqrt(-1+a**2)))/(2*(-1+a**2)**(5/2))),
(diff(Lz, ey), (a*(-3*a*sqrt(-1+a**2))+(1+2*a**2)*log(a+sqrt(-1+a**2)))/(2*(-1+a**2)**(5/2))),
(diff(Lz, ez), (a*(-3*a*sqrt(-1+a**2))+(1+2*a**2)*log(a+sqrt(-1+a**2)))/((-1+a**2)**(5/2)))
]

d2replace = [
(diff(Lx, ex, 2), a*(-15*a*sqrt(-1+a**2)*(1+2*a**2)+(1+28*a**2+16*a**4)*log(a+sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2))),
(diff(Lx, ey, 2), -2*a*(-15*a*sqrt(-1+a**2)*(1+2*a**2)+(1+28*a**2+16*a**4)*log(a+sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2))),
(diff(Lx, ez, 2), -a*(-5*a*sqrt(-1+a**2)*(2+a**2)+(2+11*a**2+2*a**4)*log(a+sqrt(-1+a**2)))/(2*(-1+a**2)**(7/2))),
(diff(Lx, ex, ey), -(1/2)*(-hyper([2,5/2],[7/2],1-1/a**2)/(10*a**4*(-1+a**2))+(sqrt(-1+a**2)*(4+a**2-14*a**4)+a*(-7+8*a**2+8*a**4)*log(a+sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2)))),    
(diff(Lx, ex, ez), 2*a*(-15*a*sqrt(-1+a**2)*(1+2*a**2)+(1+28*a**2+16*a**4)*log(a+sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2))),
(diff(Lx, ey, ez), -a*(a*sqrt(-1+a**2)*(-11-6*a**2+2*a**4)+(1+16*a**2-2*a**4)*asin(sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2)) - 1/10*a**3*hyper([3/2,5/2],[7/2],1-a**2) + 2/7*a**3*hyper([3/2,7/2],[9/2],1-a**2)),
(diff(Ly, ex, 2), -2*a*(-15*a*sqrt(-1+a**2)*(1+2*a**2)+(1+28*a**2+16*a**4)*log(a+sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2))),
(diff(Ly, ey, 2), a*(-15*a*sqrt(-1+a**2)*(1+2*a**2)+(1+28*a**2+16*a**4)*log(a+sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2))),
(diff(Ly, ez, 2), -a*(-5*a*sqrt(-1+a**2)*(2+a**2)+(2+11*a**2+2*a**4)*log(a+sqrt(-1+a**2)))/(2*(-1+a**2)**(7/2))),
(diff(Ly, ex, ey), -(1/2)*(-hyper([2,5/2],[7/2],1-1/a**2)/(10*a**4*(-1+a**2))+(sqrt(-1+a**2)*(4+a**2-14*a**4)+a*(-7+8*a**2+8*a**4)*log(a+sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2)))),
(diff(Ly, ex, ez), -a*(a*sqrt(-1+a**2)*(-11-6*a**2+2*a**4)+(1+16*a**2-2*a**4)*asin(sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2)) - 1/10*a**3*hyper([3/2,5/2],[7/2],1-a**2) + 2/7*a**3*hyper([3/2,7/2],[9/2],1-a**2)),
(diff(Ly, ey, ez), 2*a*(-15*a*sqrt(-1+a**2)*(1+2*a**2)+(1+28*a**2+16*a**4)*log(a+sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2))),
(diff(Lz, ex, 2), a*(-15*a*sqrt(-1+a**2)*(1+2*a**2)+(1+28*a**2+16*a**4)*log(a+sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2))),
(diff(Lz, ey, 2), a*(-15*a*sqrt(-1+a**2)*(1+2*a**2)+(1+28*a**2+16*a**4)*log(a+sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2))),
(diff(Lz, ez, 2), a*(-5*a*sqrt(-1+a**2)*(2+a**2)+(2+11*a**2+2*a**4)*log(a+sqrt(-1+a**2)))/(-1+a**2)**(7/2)),
(diff(Lz, ex, ey), -hyper([2,5/2],[7/2],1-1/a**2)/(10*a**4*(-1+a**2)) + (sqrt(-1+a**2)*(4+a**2-14*a**4)+a*(-7+8*a**2+8*a**4)*log(a+sqrt(-1+a**2)))/(8*(-1+a**2)**(7/2))),
(diff(Lz, ex, ez), a*(a*sqrt(-1+a**2)*(7+8*a**2)-(1+10*a**2+4*a**4)*log(a+sqrt(-1+a**2)))/(2*(-1+a**2)**(7/2))),
(diff(Lz, ey, ez), a*(a*sqrt(-1+a**2)*(7+8*a**2)-(1+10*a**2+4*a**4)*log(a+sqrt(-1+a**2)))/(2*(-1+a**2)**(7/2)))
]

dF1 = diff(FTI, ex) + diff(FTI, ey) + diff(FTI, ez)
dF2 = diff(dF1, ex) + diff(dF1, ey) + diff(dF1, ez)
dF1 = dF1.subs(d1replace)
dF1 = dF1.subs(Lreplace)
dF2 = dF2.subs(d2replace)
dF2 = dF2.subs(d1replace)
dF2 = dF2.subs(Lreplace)
'''TODO: sub in t for a eventually .subs([(a, t)])'''

Trying to speed up inversion of P matrix

In [4]:
import os
import sys
import psutil
import signal

import numpy
import sympy
import mpmath
import itertools
import pandas as pd
import multiprocessing as mp
from datetime import datetime

import MAE_funcs

from sympy import *
from sympy.abc import *

from scipy.optimize import minimize, fsolve

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

params = {'H': [], 'Ms': [], 'X': [], 't': [], 'td': [], 'f': [], 'nu': [], 'g': []}
params['H']  = [10**((i-4)*2) for i in range(0, 12+1, 2)]
params['Ms'] = [10**i for i in range(1,3+1)]
params['X']  = [1000**(i-2) for i in range(1,3+1)]
params['t']  = [(0.7057 - 0.99*i)/(-29.4286 + i) for i in range(1, 29+1, 2)]
params['td'] = [(0.7057 - 0.99*i)/(-29.4286 + i) for i in range(1, 29+1, 2)]
params['f']  = [0.005+(i-1)*.07 for i in range(1, 15+1, 2)]
params['nu'] = [.245*i-.24 for i in range(1, 15+1, 2)]
params['g']  = [10**7]
paramList = list(itertools.product(params['H'], params['Ms'], params['X'],  params['t'], params['td'], params['f'], params['nu'], params['g']))

params = {'H': [], 'Ms': [], 'X': [], 't': [], 'td': [], 'f': [], 'nu': [], 'g': []}
_keys = list(params.keys())
for idx in range(len(paramList[0])):
    params[_keys[idx]] = [_value[idx] for _value in paramList]
paramDF = pd.DataFrame(data=params)



def FreeEnergyTI(param):
    def _h0(t):
        if t<1:
            h0 = (t*(numpy.arccos(t)-t*(1-t**2)**(1/2))) / (1-t**2)**(3/2)
        if t>1:
            h0 = (re(t*(-numpy.arccosh(t)+t*(t**2-1)**(1/2))) / (-1+t**2)**(3/2))
        if td == 1:
            h0 = 2/3
        return(h0)

    def _h(td):
        if td<1:
            h  = (td*(numpy.arccos(td)-td*(1-td**2)**(1/2))) / (1-td**2)**(3/2)
        if td>1:
            h  = (re(td*(-numpy.arccosh(td)+td*(td**2 -1)**(1/2))) / (-1+td**2)**(3/2))
        if td == 1:
            h  = 2/3
        return(h)

    def Mag(EH):
        #Magnitization depends on effective H (EH)
        Mag = Ms * sympy.tanh(X * EH / Ms)
        return Mag

    def EH3():
        #finding root of given EH equation at H
        #we format into mpmath to use its findroot function
        _ex = 0
        _ey = 0
        _ez = 0
        eReplace = [(ex, _ex), (ey, _ey), (ez, _ez)]
        EH_eq = H-D*Mag(EH) + f*replacedLz.subs(eReplace)*Mag(EH) / ((1+_ex)*(1+_ey)*(1+_ez))
        #This method lets us start at H in solving, may more accurate, but it is slower I think.
        #lam_EH3 = lambdify(EH, EH_eq, 'scipy')
        #EH3 = fsolve(lam_EH3, H)
        EH3 = sympy.solvers.solve(EH_eq)
        #EH3 = mpmath.findroot(lam_EH3, H, solver = 'mdnewton', norm = 'max-norm')
        if len(EH3) > 1:
            print('Warning, solving for EH3 gives more than 1 solution: ' + str(EH3))
            raise
        EH3 = float(EH3[0])
        print('\t H:   ' + str(H))
        print('\t EH3: ' + str(EH3))
        return EH3

    #TI: transverse isotropy
    #H, Ms, X, t, td, f, nu, g, h, h0 = symbols('H, Ms, X, t, td, f, nu, g, h, h0', real = True)
    #z, Sigma, g, nu, k, l, n, m, p, k1, h0, h, t,td, kp, lp, np, mp, pp, kd, ld, nd, md, pd, L1, Pi, Pd, P, Ltilde, ex, ey, ez, a, f, H, D, M, e, C, Ms, MTI, EH, Lz, X, ax, ay, az = symbols('z, Sigma g nu k l n m p k1 h0 h t td kp lp np mp pp kd ld nd md pd L1 Pi Pd P Ltilde ex ey ez a f H D M e C Ms MTI EH Lz X ax ay az', real=True)
    print('defining params')
    param = list(param)
    H  = param[0]
    Ms = param[1]
    X  = param[2]
    t  = param[3]
    td = param[4]
    f  = param[5]
    nu = param[6]
    g  = param[7]
    h  = _h(td)
    h0 = _h0(t)
    
    print('defining variables')
    Sigma = 2*g*(1+nu)
    k     = Sigma / (2*(1+nu)*(1-2*nu))
    l     = (Sigma*nu) / (2*(1+nu)*(1-2*nu))
    n     = Sigma*(1-nu) / (2*(1+nu)*(1-2*nu))
    m     = Sigma / (2*(1+nu))
    p     = Sigma / (2*(1+nu))
    k1    = 2*g*(1+nu) / (1-2*nu)
    kp    = ((7*h0-2*t**2-4*t**2*h0)*g+3*(h0-2*t**2+2*t**2*h0)*k1) / (8*(1-t**2)*g*(4*g+3*k1))
    lp    = ((g+3*k1)*(2*t**2-h0-2*t**2*h0)) / (4*(1-t**2)*g*(4*g+3*k1))
    np    = ((6-5*h0-8*t**2+8*t**2*h0)*g+3*(h0-2*t**2+2*t**2*h0)*k1) / (2*(1-t**2)*g*(4*g+3*k1))
    mp    = ((15*h0-2*t**2-12*t**2*h0)*g+3*(3*h0-2*t**2)*k1) / (16*(1-t**2)*g*(4*g+3*k1))
    pp    = (2*(4-3*h0-2*t**2)*g+3*(2-3*h0+2*t**2-3*t**2*h0)*k1) / (8*(1-t**2)*g*(4*g+3*k1))
    kd    = ((7*h-2*td**2-4*td**2*h)*g+3*(h-2*td**2+2*td**2*h)*k1) / (8*(1-td**2)*g*(4*g+3*k1))
    ld    = ((g+3*k1)*(2*td**2-h-2*td**2*h)) / (4*(1-td**2)*g*(4*g+3*k1))
    nd    = ((6-5*h-8*td**2+8*td**2*h)*g+3*(h-2*td**2+2*td**2*h)*k1) / (2*(1-td**2)*g*(4*g+3*k1))
    md    = ((15*h-2*td**2-12*td**2*h)*g+3*(3*h-2*td**2)*k1) / (16*(1-td**2)*g*(4*g+3*k1))
    pd    = (2*(4-3*h-2*td**2)*g+3*(2-3*h+2*td**2-3*td**2*h)*k1) / (8*(1-td**2)*g*(4*g+3*k1))

    ax,ay,az,ex,ey,ez,EH=symbols('ax ay az ex ey ez EH',real=True)
    #ax = td*((1+ex)/(1+ez))
    #ay = td*((1+ey)/(1+ez))
    #az = td*((1+ez)/(1+ex))
    #     Lx = (1-1/(1-ax**2)*(1-ax/sqrt(1-ax**2)*asin(sqrt(1-ax**2)))/2)
    #     Ly = (1-1/(1-ay**2)*(1-ay/sqrt(1-ay**2)*asin(sqrt(1-ay**2)))/2)
    #     Lz = (1 /(1-az**2)*(1-az/sqrt(1-az**2)*asin(sqrt(1-az**2))))
    #     #import pdb; pdb.set_trace()
    #     Lx = Lx.subs([(ax,td*((1+ex)/(1+ez)))])
    #     Ly = Ly.subs([(ay,td*((1+ey)/(1+ez)))])
    #     Lz = Lz.subs([(az,td*((1+ez)/(1+ex)))])

    Lx = Function('Lx')(ex, ey, ez)
    Ly = Function('Ly')(ex, ey, ez)
    Lz = Function('Lz')(ex, ey, ez)
    Lreplace = [
    (Lx, 1-1/(1-ax**2)*(1-ax/sqrt(1-ax**2)*asin(sqrt(1-ax**2)))/2),
    (Ly, 1-1/(1-ay**2)*(1-ay/sqrt(1-ay**2)*asin(sqrt(1-ay**2)))/2),
    (Lz, 1/(1-az**2)*(1-az/sqrt(1-az**2)*asin(sqrt(1-az**2))))
    ]
    aspectReplace = [(ax, td*((1+ex)/(1+ez))),
    (ay, td*((1+ey)/(1+ez))),
    (az, td*((1+ez)/(1+ex)))
    ]
    #We preserve L's for derivatives later, alser calculating L's this way makes minimization faster (idk why)
    replacedLx = Lz.subs(Lreplace).subs(aspectReplace)
    replacedLy = Lz.subs(Lreplace).subs(aspectReplace)
    replacedLz = Lz.subs(Lreplace).subs(aspectReplace)

    print('defining matrices')
    L1 = numpy.matrix([[k1+(4*g)/3, k1-(2*g)/3, k1-(2*g)/3, 0, 0, 0], [k1-2*g/3, k1+4*g/3, k1-2*g/3, 0, 0, 0], [k1-2*g/3, k1-2*g/3, k1+4*g/3, 0, 0, 0], [0, 0, 0, g, 0, 0], [0, 0, 0, 0, g, 0], [0, 0, 0, 0, 0, g]])
    Pi = numpy.matrix([[kp+mp, kp-mp, lp, 0, 0, 0], [kp-mp, kp+mp, lp, 0, 0, 0], [lp, lp, np, 0, 0, 0], [0, 0, 0, 2*pp, 0, 0], [0, 0, 0, 0, 2*pp, 0], [0, 0, 0, 0, 0, 2*mp]])
    Pd = numpy.matrix([[kd+md, kd-md, ld, 0, 0, 0], [kd-md, kd+md, ld, 0, 0, 0], [ld, ld, nd, 0, 0, 0], [0, 0, 0, 2*pd, 0, 0], [0, 0, 0, 0, 2*pd, 0], [0, 0, 0, 0, 0, 2*md]])
    P = 1/(1-f)*(Pi-f*Pd)
    Ltilde = L1+f/(1-f)*numpy.linalg.inv(P)
    
    D = re((4*numpy.pi)/(t**2-1)*(t/(2*(t**2-1)**(1/2))*log((t+(t**2-1)**(1/2))/(t-(t**2-1)**(1/2)))-1))
    C = numpy.matrix([[Ltilde[0,0], Ltilde[0,1], Ltilde[0,2]], [Ltilde[1,0], Ltilde[1,1], Ltilde[1,2]], [Ltilde[2,0], Ltilde[2,1], Ltilde[2,2]]])
    MTI = (sympy.Matrix([[ex, ey, ez]]) * C * sympy.Matrix([[ex], [ey], [ez]]))[0]
    
    #Equation for free energy equation in transverse isotropy. Reference NIST notebook for equation.
    MagEH3 = Mag(EH3())
    Ls = ((f*D*MagEH3**2)/2)*((replacedLx + replacedLy + replacedLz)/((1+ex)*(1+ey)*(1+ez)))
    LsMTI = ((f*D*MagEH3**2)/2)*((replacedLx + replacedLy + replacedLz)/((1+ex)*(1+ey)*(1+ez))) + MTI/2
    FTI = -f*H*MagEH3 + ((f*D*MagEH3**2)/2)*((replacedLx + replacedLy + replacedLz)/((1+ex)*(1+ey)*(1+ez))) + MTI/2

    print('minimizing')
    strains = [ex, ey, ez]
    FTI_lam = lambdify(strains, FTI, modules = 'numpy')
    def vectorizeFTI(_list):
        return FTI_lam(_list[0], _list[1], _list[2])
    strains0 = numpy.array([0,0,0])
    minimization = minimize(fun = vectorizeFTI, x0 = strains0, method = 'Nelder-Mead');

    return minimization.x


In [5]:
start = datetime.now()
FreeEnergyTI(paramList[0])
print(datetime.now() - start)

defining params
defining variables
defining matrices
	 H:   1e-08
	 EH3: 8.086342340341222e-07
minimizing
0:00:00.068063


In [2]:
start = datetime.now()
FreeEnergyTI(paramList[0])
print(datetime.now() - start)

defining params
defining variables
defining matrices
H:   1e-08
EH3: 8.086342340341222e-07
minimizing
0:00:00.337818


In [21]:
for i in range(10):
    start = datetime.now()
    FreeEnergyTI(paramList[i])
    end = datetime.now() - start
    print(end)
    print('')

defining params
0:00:00
defining variables
0:00:00.001000
defining matrices
0:00:00.083076
minimizing
	0:00:00
	0:00:00
	0:00:00.003003
0:00:00.020018
0:00:00.104094

defining params
0:00:00
defining variables
0:00:00
defining matrices
0:00:00.086079
minimizing
	0:00:00
	0:00:00
	0:00:00.004004
0:00:00.021019
0:00:00.108099

defining params
0:00:00
defining variables
0:00:00
defining matrices
0:00:00.085077
minimizing
	0:00:00
	0:00:00
	0:00:00.026024
0:00:00.045041
0:00:00.131119

defining params
0:00:00
defining variables
0:00:00.001001
defining matrices
0:00:00.086078
minimizing
	0:00:00
	0:00:00
	0:00:00.023021
0:00:00.042039
0:00:00.129118

defining params
0:00:00
defining variables
0:00:00.001001
defining matrices
0:00:00.085077
minimizing
	0:00:00
	0:00:00
	0:00:00.026024
0:00:00.044040
0:00:00.130118

defining params
0:00:00
defining variables
0:00:00
defining matrices
0:00:00.085077
minimizing
	0:00:00
	0:00:00
	0:00:00.024022
0:00:00.044040
0:00:00.130118

defining params
0:0